In [ ]:
import numpy as np
import math
import random
from random import shuffle
import matplotlib.pyplot as plt
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
cvxopt_solvers.options['show_progress'] = False

#Polynomial Kernel
def kernel(x, y, order):
    return (1 + np.dot(x, y)) ** order

#Training Function
def train(X, y, classes, C, order):
  m,n = X.shape
  params = {}

  #Kernel Matrix
  H = np.zeros((m,m))
  for i in range(m):
    for j in range(m):
      H[i,j] = kernel(X[i],X[j],order)
  #print("H Matrix done")
  
  for i in range(classes):
    #CVXOPT Initialization
    yc = y/(i+1)
    yc[yc != 1] = -1
    #print(yc[:200])
    P = cvxopt_matrix(np.outer(yc,yc) * H)
    q = cvxopt_matrix(-np.ones((m, 1)))
    G = cvxopt_matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
    h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * C)))
    A = cvxopt_matrix(yc.reshape(1, -1))
    b = cvxopt_matrix(np.zeros(1))

    #Run Solver
    svm_h = cvxopt_solvers.qp(P, q, G, h, A, b)
    alphas = np.ravel(svm_h['x'])

    #All SVs
    S = (alphas > 1e-9)
    y_sv = yc[S]
    alphas_sv = alphas[S]
    X_sv = X[S]
    #print(len(alphas_sv))

    #Marginal SVs
    T = ((alphas > 1e-9) &(alphas < (C-1e-4)))
    y_msv = yc[T]
    alphas_msv = alphas[T]
    X_msv = X[T]
    #print(len(alphas_msv))

    #bias calculation
    b = 0
    for a in range(len(alphas_msv)):
      sum = 0
      for s in range(len(alphas_sv)):
        sum = sum + alphas_sv[s] * y_sv[s] * kernel(X_msv[a],X_sv[s],order)
      b = b + y_msv[a] - sum
    b = b/len(alphas_msv)

    #Parameters for class i+1
    params['alphas_sv' + str(i+1)] = alphas_sv
    params['X_sv' + str(i+1)] = X_sv
    params['y_sv' + str(i+1)] = y_sv
    params['b' + str(i+1)] = b

    #print("Number of Classes Completed:", i+1)

  return(params)

#Prediction Function
def predict(X, params, classes, order):
  score_set = np.zeros((classes)) #

  for i in range(classes):
    alphas_sv = params['alphas_sv' + str(i+1)]
    X_sv = params['X_sv' + str(i+1)]
    y_sv = params['y_sv' + str(i+1)]
    b = params['b' + str(i+1)]

    sum = 0
    for s in range(len(alphas_sv)):
      sum += alphas_sv[s] * y_sv[s] * kernel(X, X_sv[s],order)
    sum += b
    score_set[i] = sum

  prediction = np.argmax(score_set) + 1.
  return(prediction)

#Train Set
X_train1 = np.load('/content/drive/My Drive/Colab Notebooks/8 class SVM/X_train.npy')
Y_train1 = np.load('/content/drive/My Drive/Colab Notebooks/8 class SVM/Y_train.npy')
index = [i for i in range(len(X_train1))]
shuffle(index)
X_train1 = X_train1[index]
Y_train1 = Y_train1[index]
Y_train1 = Y_train1 + 1

#Test Set 2
X_test2 = np.load('/content/drive/My Drive/Colab Notebooks/8 class SVM/X_test2.npy')
Y_test2 = np.load('/content/drive/My Drive/Colab Notebooks/8 class SVM/Y_test2.npy')
index = [i for i in range(len(X_test2))]
shuffle(index)
X_test2 = X_test2[index]
Y_test2 = Y_test2[index]
Y_test2 = Y_test2 + 1

#Test Set 1
X_test1 = np.load('/content/drive/My Drive/Colab Notebooks/8 class SVM/X_test.npy')
Y_test1 = np.load('/content/drive/My Drive/Colab Notebooks/8 class SVM/Y_test.npy')
Y_test1  = Y_test1 + 1

#Final Training and Testing Data
X_train = np.vstack((X_train1[:200],X_test2[:200]))
Y_train = np.hstack((Y_train1[:200],Y_test2[:200]))

X_test = np.vstack((X_test1,X_test2))
Y_test = np.hstack((Y_test1,Y_test2))

#Target Function
def functionopt(C, order, classes = 8, X = X_train, Y = Y_train, X_test = X_test, Y_test = Y_test):
  #Training
  parameters = train(X,Y,classes,C,order)

  #Prediction
  y_predict = np.zeros(len(X_test))
  for i in range(len(X_test)):
    y_predict[i] = predict(X_test[i],parameters,classes,order)
  
  correct = np.sum(y_predict == Y_test)
  Accuracy = (correct)/len(y_predict)

  #Recall of Class 1
  check = (Y_test == 1)
  y_predict_1 = y_predict[check]
  Recall = np.sum(y_predict_1 == 1)/len(y_predict_1)
  return Accuracy,Recall

#Function to find index of a list
def index_search(a,list):
    for i in range(len(list)):
        if list[i] == a:
            return i
    return -1

#Function to sort by values
def sort_by_values(list, values):
    sorted_list = []
    while(len(sorted_list)!=len(list)):
        if index_search(min(values),values) in list:
            sorted_list.append(index_search(min(values),values))
        values[index_search(min(values),values)] = math.inf
    return sorted_list

#NSGA-2 non dominated sort
def non_dominated_sort(values1, values2):
    S=[[] for i in range(len(values1))]
    front = [[]]
    n=[0 for i in range(len(values1))]

    for p in range(len(values1)):
        for q in range(len(values1)):
            if (values1[p] > values1[q] and values2[p] > values2[q]) or (values1[p] >= values1[q] and values2[p] > values2[q]) or (values1[p] > values1[q] and values2[p] >= values2[q]):
                if q not in S[p]:
                    S[p].append(q)
            elif (values1[q] > values1[p] and values2[q] > values2[p]) or (values1[q] >= values1[p] and values2[q] > values2[p]) or (values1[q] > values1[p] and values2[q] >= values2[p]):
                n[p] = n[p] + 1
        if n[p]==0:
            if p not in front[0]:
                front[0].append(p)
    
    i = 0
    while(front[i] != []):
        Q=[]
        for p in front[i]:
            for q in S[p]:
                n[q] =n[q] - 1
                if( n[q]==0):
                    if q not in Q:
                        Q.append(q)
        i = i+1
        front.append(Q)
    del front[len(front)-1]
    return front

#Crowding distance
def crowding_distance(values1, values2, front):
    distance = [0 for i in range(len(front))]
    sorted1 = sort_by_values(front, values1[:])
    sorted2 = sort_by_values(front, values2[:])
    #print(values2)
    for i in range(len(front)):
      if (front[i] == sorted1[0]) or (front[i] == sorted2[0]) or (front[i] == sorted1[-1]) or (front[i] == sorted2[-1]):
        distance[i] = 1000
      else:
        distance[i] += (values1[sorted1[index_search(front[i],sorted1)+1]] - values1[sorted1[index_search(front[i],sorted1)-1]])/(max(values1)-min(values1))
        distance[i] += (values2[sorted2[index_search(front[i],sorted2)+1]] - values2[sorted2[index_search(front[i],sorted2)-1]])/(max(values2)-min(values2))
    return distance

#Function to carry out the crossover
def crossover(a,b):
    offspring = []
    #Crossover for C
    r = random.random()
    if r>0.4:
      offspring.append((a[0]+b[0])/2)
    else:
      offspring.append(mutation(a[0],b[0]))
    #Crossover for order
    r = random.random() 
    if r>0.5:
        offspring.append(a[1])
    else:
        offspring.append(b[1])
    return offspring

#Mutation for C
def mutation(a,b):
  mutatedC = ((a+b)/2) * (random.random()) * 2
  if mutatedC > 1:
    return mutatedC
  else:
    return 1

#Initilisation of first generation
C = [5.0,15.0,25.0,35.0,45.0,55.0,65.0,75.0]
order = [2,3,4]
pop_size = len(C)*len(order)
Pop = []
for i in order:
  for j in C:
    Pop.append([j,i]) #Each Pair: [C,order]

Full_Pop = Pop[:] #Full_Pop includes offspring of Pop
for i in range(pop_size):
  offspring = crossover(Pop[pop_size-i-2],Pop[pop_size-i-1])
  if offspring not in Full_Pop:
    Full_Pop.append(offspring)

while len(Full_Pop) != 2*pop_size: #To complete first population
  offspring = crossover(Full_Pop[random.randint(0,len(Full_Pop)-1)],Full_Pop[random.randint(0,len(Full_Pop)-1)])
  if offspring not in Full_Pop:
    Full_Pop.append(offspring)
#print(Full_Pop)

#Main Loop
max_generations = 4
gen_count = 0
while gen_count < max_generations:
  #Optimization Metric Calculation
  Accuracy_values = []
  Recall_values = []
  for i in range(len(Full_Pop)):
    A,R = functionopt(Full_Pop[i][0], Full_Pop[i][1])
    Accuracy_values.append(A)
    Recall_values.append(R)
  
  #Obtaining Fronts using Optimization Metrics
  non_dominated_sorted_fronts = non_dominated_sort(Accuracy_values[:],Recall_values[:])
  #print(non_dominated_sorted_fronts)
  
  #Calculation of Crowding Distance
  crowding_distance_values=[]
  for i in range(len(non_dominated_sorted_fronts)):
    crowding_distance_values.append(crowding_distance(Accuracy_values[:],Recall_values[:],non_dominated_sorted_fronts[i][:]))
  #print(crowding_distance_values)

  gen_count += 1
  print("The best pairs of Generation",gen_count,"are")
  for pair in non_dominated_sorted_fronts[0]:
      print(Full_Pop[pair], ", with Accuracy",Accuracy_values[pair]*100,"%, with Recall for Class-1",Recall_values[pair]*100,"%")
  print("\n")

  #Creating New Population
  new_Poplist = []  #List of pairs to be taken from previous generation
  for i in range(len(non_dominated_sorted_fronts)):
    front_i_index = [j for j in range(len(non_dominated_sorted_fronts[i]))]   #List of indices of the pairs in each front
    dist_sorted_front_i_index = sort_by_values(front_i_index[:], crowding_distance_values[i][:])    #Sorting above list using Crowding Distance of that front
    front_ascending_distance = [non_dominated_sorted_fronts[i][dist_sorted_front_i_index[j]] for j in range(len(non_dominated_sorted_fronts[i]))]    
    front_ascending_distance.reverse()
    front_descending_distance = front_ascending_distance[:] #List of pairs from previous generation within each front arranged in descending order w.r.t. Crowding Distance
    for pair_number in front_descending_distance:
      new_Poplist.append(pair_number)
      if (len(new_Poplist)==pop_size):
        break
    if (len(new_Poplist) == pop_size): #We only take best half of previous generation
      break

  Pop = [Full_Pop[i] for i in new_Poplist]  
  Full_Pop = Pop[:] #Full_Pop2 contains new Pop and it's offspring
  for i in range(math.ceil(pop_size/2)):
    offspring = crossover(Pop[random.randint(0,math.ceil(len(Pop)/2))],Pop[random.randint(0,math.ceil(len(Pop)/2))])
    if offspring not in Full_Pop:
      Full_Pop.append(offspring)

  while len(Full_Pop) != 2*pop_size: #To complete population
    offspring = crossover(Full_Pop[random.randint(0,len(Full_Pop)-1)],Full_Pop[random.randint(0,len(Full_Pop)-1)])
    if offspring not in Full_Pop:
      Full_Pop.append(offspring)
  #print(Full_Pop)

#Final Population
print("The Final Population is:")
print(Pop)